In [1]:
%run header.ipynb
print(paths)

dryrun = True

from IPython.display import clear_output
sdsobj = SDS.SDSobj(paths['SDS_TOP'], sds_type='D', format='MSEED')

# Parse lookuptable and convert good CSV files to SDS
lookuptableDF = LLE.removed_unnamed_columns(pd.read_csv(paths['lookuptable']))
lookuptableDF = lookuptableDF.sort_values(by=['starttime'])
print("lookuptableDF = \n", lookuptableDF.columns)
print(lookuptableDF)

{'HOME': '/home/thompsong', 'Developer': '/home/thompsong/Developer', 'tremor_explorer_lib': '/home/thompsong/Developer/tremorExplorer/lib', 'src': '/home/thompsong/Developer/kitchensinkGT/PROJECTS/ROCKETSEIS/launchpad_erosion', 'work': '/home/thompsong/work', 'outdir': '/home/thompsong/work/PROJECTS/KSC_EROSION', 'DROPBOX_TOP': '/home/thompsong/Dropbox', 'CORRECTED': '/home/thompsong/work/PROJECTS/KSC_EROSION/corrected', 'lookuptable': '/home/thompsong/work/PROJECTS/KSC_EROSION/corrected/lookuptable.csv', 'transducersCSVfile': '/home/thompsong/work/PROJECTS/KSC_EROSION/transducer_metadata.csv', 'CONFIGDIR': '/home/thompsong/Developer/tremorExplorer/config', 'SDS_TOP': '/shares/hal9000/RAIDZ/iceweb_products/SDS', 'RSAM_DIR': '/shares/hal9000/RAIDZ/iceweb_products/RSAM'}
lookuptableDF = 
 Index(['sourcefile', 'outputfile', 'starttime', 'endtime', 'hours', 'npts',
       'nRECS', 'Fs', 'numValidTimes', 'numTimeSkips', 'passed', 'SDS'],
      dtype='object')
                              

In [ ]:
transducersDF = LLE.removed_unnamed_columns(pd.read_csv(paths['transducersCSVfile']))
print("transducersDF = \n",transducersDF.columns)

lookuptableDF['SDS'] = False

sds_column_exists = False
if 'SDS' in lookuptableDF.columns:
    sds_column_exists = True

for index, row in lookuptableDF.iterrows():
    print(row)
    row_to_sds_done = False
    if row['passed']: 
        successful = False
        if sds_column_exists:
            row_to_sds_done = row['SDS']
        if not row_to_sds_done:
            df2 = pd.read_csv(os.path.join(paths['CORRECTED'],row['outputfile']))
            print(f"- writing {row['outputfile']} to SDS")
            print("row = \n ",row)
            #print("sdsobj = \n", sdsobj)
            if row['passed']:
                successful = LLE.convert2sds(df2, sdsobj, transducersDF, dryrun=dryrun)
            else:
                try:
                    successful = LLE.convert2sds_badfile(df2, sdsobj, transducersDF, dryrun=dryrun)
                except:
                    print('Failed to convert bad CSV file')
            lookuptableDF.at[index,'SDS'] = successful
            if successful and not dryrun:
                lookuptableDF.to_csv(lookuptable)     
    clear_output(wait=True)
del sdsobj

sourcefile       20220923\100hz\100hz_Sensors_100Hz102.csv
outputfile       20220923.100hz.100hz_Sensors_100Hz102.csv
starttime                              2022/08/30 12:00:00
endtime                                2022/08/30 16:00:00
hours                                                  4.0
npts                                               1439604
nRECS                                            1439604.0
Fs                                              100.000095
numValidTimes                                      1439604
numTimeSkips                                           4.0
passed                                                True
SDS                                                  False
Name: 794, dtype: object
